In [ ]:
import torch
import os
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from moviepy.editor import AudioFileClip

In [ ]:
# Unzip Data
!tar -xf MELD.Raw.tar.gz

# Unzip Train data
!tar -xf ./MELD.Raw/train.tar.gz
# Unzip Valid data
!tar -xf ./MELD.Raw/dev.tar.gz
# Unzip Test data
!tar -xf ./MELD.Raw/test.tar.gz

In [ ]:
import librosa
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import os
import json
import math
from sklearn.preprocessing import LabelEncoder,StandardScaler
import csv
from sklearn.model_selection import train_test_split

In [ ]:
!git clone https://github.com/declare-lab/MELD

Cloning into 'MELD'...
remote: Enumerating objects: 493, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 493 (delta 10), reused 0 (delta 0), pack-reused 475
Receiving objects: 100% (493/493), 8.12 MiB | 15.28 MiB/s, done.
Resolving deltas: 100% (258/258), done.


In [ ]:
train_data=pd.read_csv('/content/drive/MyDrive/MELD/MELD.Raw/train_meld_emo.csv', encoding='cp949')
valid_data=pd.read_csv('/content/drive/MyDrive/MELD/MELD.Raw/dev_meld_emo.csv', encoding='cp949')
test_data=pd.read_csv('/content/drive/MyDrive/MELD/MELD.Raw/test_meld_emo.csv', encoding='cp949')

In [ ]:
test_data2 = pd.read_csv('/content/drive/MyDrive/MELD/MELD.Raw/test_sent_emo.csv')

In [ ]:
y_train=train_data['Emotion']
y_valid=valid_data['Emotion']
y_test=test_data['Emotion']

In [ ]:
data1 = pd.read_csv("/content/drive/MyDrive/MELD/MELD.Raw/train_meld_emo.csv",encoding = 'cp949')
data2=pd.read_csv("/content/drive/MyDrive/MELD/MELD.Raw/test_meld_emo.csv", encoding = 'cp949')

In [ ]:
test_data2.head(-1)

,Sr No.,Utterance,Speaker,Emotion,Sentiment,Dialogue_ID,Utterance_ID,Season,Episode,StartTime,EndTime
0,1,Why do all youre coffee mugs have numbers on ...,Mark,surprise,positive,0,0,3,19,"00:14:38,127","00:14:40,378"
1,2,Oh. Thats so Monica can keep track. That way ...,Rachel,anger,negative,0,1,3,19,"00:14:40,629","00:14:47,385"
2,3,Y'know what?,Rachel,neutral,neutral,0,2,3,19,"00:14:56,353","00:14:57,520"
3,19,"Come on, Lydia, you can do it.",Joey,neutral,neutral,1,0,1,23,"0:10:44,769","0:10:46,146"
4,20,Push!,Joey,joy,positive,1,1,1,23,"0:10:46,146","0:10:46,833"
...,...,...,...,...,...,...,...,...,...,...,...
2604,2759,"Yeah, fade the accent out and people will thin...",Monica,anger,negative,279,10,6,4,"00:14:28,200","00:14:35,039"
2605,2760,"Yeah, I mean, come on Ross, no one will even n...",Rachel,neutral,neutral,279,11,6,4,"00:14:35,457","00:14:40,211"
2606,2761,They’re not listening too me?,Ross,surprise,negative,279,12,6,4,"00:14:42,256","00:14:43,840"
2607,2762,Of course they’re listening to you! Everybody ...,Rachel,neutral,neutral,279,13,6,4,"00:14:44,008","00:14:48,511"


In [ ]:
!pip install torch torchvision torchaudio transformers google-colab opencv-python-headless

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.9 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline


In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

image_captioner = pipeline("image-to-text", model="Abdou/vit-swin-base-224-gpt2-image-captioning", device=0) # infer the caption

  warnings.warn(

The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(

Some weights of the model checkpoint at Abdou/vit-swin-base-224-gpt2-image-captioning were not used when initializing VisionEncoderDecoderModel: ['decoder.transformer.h.3.attn.bias', 'decoder.transformer.h.5.attn.bias', 'decoder.transformer.h.11.attn.masked_bias', 'decoder.transformer.h.6.crossattention.masked_bias', 'decoder.transformer.h.7.attn.masked_bias', 'decoder.transformer.h.8.crossattention.bias', 'decoder.transformer.h.4.crossattention.masked_bias', 'decoder.transformer.h.1.crossattention.masked_bias', 'decoder.transformer.h.7.attn.bias

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchaudio
from transformers import BertTokenizer
from google.colab.patches import cv2_imshow

import cv2
import os
import numpy as np
import pandas as pd
from transformers import pipeline
from PIL import Image

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class MELDDataset(Dataset):
    def __init__(self, csv, path, transform=None, max_video_len=30, max_audio_len=16000, max_text_len=128):
        self.df = pd.read_csv(csv)
        self.label = {self.df['Emotion'].unique()[i]: i for i in range(len(self.df['Emotion'].unique()))}
        self.path = path
        self.max_video_len = max_video_len
        self.max_audio_len = max_audio_len
        self.max_text_len = max_text_len
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.video_transform = transform

    def __len__(self):
        return len(self.df)

    def load_video(self, video_path):
        cap = cv2.VideoCapture(video_path)
        frames = []
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            frame = self.video_transform(frame)
            frames.append(frame)
            if len(frames) >= self.max_video_len:
                break
        cap.release()
        if len(frames) < self.max_video_len:
            padding_frames = [frames[-1]] * (self.max_video_len - len(frames))
            frames.extend(padding_frames)
        return torch.stack(frames)

    def load_audio(self, audio_path):
        try:
            waveform, sample_rate = torchaudio.load(audio_path)
            if waveform.size(0) > 1:
                waveform = torch.mean(waveform, dim=0, keepdim=True)
            if waveform.size(1) > self.max_audio_len:
                waveform = waveform[:, :self.max_audio_len]
            else:
                padding = self.max_audio_len - waveform.size(1)
                waveform = torch.nn.functional.pad(waveform, (0, padding))
            return waveform
        except:
            return torch.zeros(1, self.max_audio_len)

    def tokenize_text(self, text):
        encoding = self.tokenizer(text, truncation=True, padding='max_length', max_length=self.max_text_len, return_tensors='pt')
        return encoding['input_ids'].squeeze(), encoding['attention_mask'].squeeze()

    def __getitem__(self, idx):
        filename = 'dia' + str(self.df.iloc[idx]['Dialogue_ID']) + '_utt' + str(self.df.iloc[idx]['Utterance_ID']) + '.mp4'
        text = self.df.iloc[idx]['Utterance'].replace('\x92', "'")

        video = self.load_video(self.path + filename)
        audio = self.load_audio(self.path + filename)
        text, attention_mask = self.tokenize_text(text)
        label = self.label[self.df.iloc[idx]['Emotion']]
        dialogue_id = self.df.iloc[idx]['Dialogue_ID']
        utterance_id = self.df.iloc[idx]['Utterance_ID']
        return video, audio, text, attention_mask, label, dialogue_id, utterance_id

def collate_fn(batch):
    videos, audios, texts, attention_masks, labels, dialogue_ids, utterance_ids = zip(*batch)
    videos = torch.stack(videos).to(device)
    audios = torch.stack(audios).to(device)
    texts = torch.stack(texts).to(device)
    attention_masks = torch.stack(attention_masks).to(device)
    labels = torch.tensor(labels).to(device)
    return texts, attention_masks, audios, videos, labels, dialogue_ids, utterance_ids

def MELD(datatype, transform=None, batch_size=4, collate=collate_fn):
    """DataLoader. \\
    Expected File structure is: \\
    ├── train\\
    ├── valid\\
    ├── test  \\
    ├── train.csv\\
    ├── valid.csv\\
    └── test.csv\\
    Change if you want. \\
    If transform is None, it just resizes data and returns Tensor.\\
    Video (Batch, Frame, Channel, Height, Width) \\
    Audio (Batch, Channel, Sample) \\
    Text  (Batch, tokenized Length)\\
    Label (Batch)
    """
    if datatype == 'train':
        csv_file = '/content/drive/MyDrive/MELD/MELD.Raw/train_sent_emo.csv'
        data_folder = '/content/drive/MyDrive/MELD/train_splits/'
    elif datatype == 'valid':
        csv_file = '/content/drive/MyDrive/MELD/MELD.Raw/dev_sent_emo.csv'
        data_folder = '/content/drive/MyDrive/MELD/dev_splits_complete/'
    elif datatype == 'test':
        csv_file = '/content/drive/MyDrive/MELD/MELD.Raw/test_sent_emo.csv'
        data_folder = '/content/drive/MyDrive/MELD/output_repeated_splits_test/'
    if transform is None:
        transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((480, 640)),
            transforms.ToTensor(),
        ])
    dataset = MELDDataset(csv_file, data_folder, transform=transform)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=collate)
    return dataloader

if __name__ == '__main__':
    dataloader = MELD('test', batch_size=16)


    captions = []
    dialogue_ids = []
    utterance_ids = []

    for batch in dataloader:
        batch_input_tokens, batch_attention_masks, batch_audio, batch_video, batch_labels, batch_dialogue_ids, batch_utterance_ids = batch
        for i in range(len(batch_video)):
            img = batch_video[i][0].permute(1, 2, 0).cpu().numpy() * 255
            pil_image = Image.fromarray(img.astype(np.uint8))
            caption = image_captioner(pil_image)[0]['generated_text']
            captions.append(caption)
            dialogue_ids.append(batch_dialogue_ids[i])
            utterance_ids.append(batch_utterance_ids[i])

    df = pd.DataFrame({
        'dialogue_id': dialogue_ids,
        'utterance_id': utterance_ids,
        'caption': captions
    })



    print(df)


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



KeyboardInterrupt: 

In [ ]:
df.rename(columns={'caption' : 'image_caption'})

,dialogue_id,utterance_id,image_caption
0,259,13,A woman in a red shirt and black pants is stan...
1,70,7,A woman sitting on a chair with a smile on her...
2,189,8,A man taking a picture of a woman taking a pic...
3,12,19,A man and woman sitting at a table with a plat...
4,166,3,Two men standing next to each other in front o...
...,...,...,...
2605,20,1,A group of people standing next to each other.
2606,21,2,A woman in a dress standing in front of a mirror.
2607,262,13,A man and woman are standing next to each other.
2608,178,5,A man holding a baby holding a hot dog.


In [ ]:
df.to_csv('/content/drive/MyDrive/captions.csv')

In [ ]:
history = pd.read_csv('/content/drive/MyDrive/Refin3_history_test.csv')

In [ ]:
df_selected = df[['dialogue_id', 'utterance_id', 'caption']]
history_selected = history[['Dialogue_ID', 'Utterance_ID']]

merged_df = pd.merge(history_selected, df_selected, left_on=['Dialogue_ID', 'Utterance_ID'], right_on=['dialogue_id', 'utterance_id'], how='left')

history['image_caption'] = merged_df['caption']

In [ ]:
history.to_csv(('/content/drive/MyDrive/img_captions.csv'))

In [ ]:
null_count = history['image_caption'].isnull().sum()
print("image_caption 열에 null 값 수:", null_count)

image_caption 열에 null 값 수: 0


In [ ]:
!pwd

/content


In [ ]:
from transformers import pipeline
from PIL import Image

image_captioner = pipeline("image-to-text", model="Abdou/vit-swin-base-224-gpt2-image-captioning")


In [ ]:
pil_image = Image.fromarray(img.astype(np.uint8))

caption = image_captioner(pil_image)[0]['generated_text']
print(f"caption: {caption}")

  warnings.warn(



caption: A man sitting at a table with a laptop computer.
